In [34]:
import numpy as np
import matplotlib.pyplot as plt
import json
from shapely.geometry import shape, MultiPolygon
from descartes import PolygonPatch
import csv
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

In [35]:
with open("petition-jun-29-8h.json") as json_file:
    petition = json.load(json_file)

total_petition_votes = 0
signatures_by_constituency = {}
for constituency in petition['data']['attributes']['signatures_by_constituency']:
    signatures_by_constituency[constituency['name']] = constituency['signature_count']
    total_petition_votes += constituency['signature_count']
    
signature_list = [(k,v) for (k,v) in signatures_by_constituency.items()]
signature_list.sort(key=lambda x: x[1])

print("Votes  Constituency")
for constituency in signature_list[:10]:
    print("%5i %s" % constituency[::-1])
print("..... .....")
for constituency in signature_list[-10:]:
    print("%5i %s" % constituency[::-1])

Votes  Constituency
  597 Na h-Eileanan an Iar
 1224 Glasgow East
 1327 Airdrie and Shotts
 1350 Caithness, Sutherland and Easter Ross
 1367 Banff and Buchan
 1373 Orkney and Shetland
 1402 Glenrothes
 1508 Motherwell and Wishaw
 1548 Walsall North
 1566 Easington
..... .....
18706 Battersea
18949 Brighton, Pavilion
19229 Hackney South and Shoreditch
19790 Islington North
19983 Richmond Park
20331 Hackney North and Stoke Newington
21307 Holborn and St Pancras
22229 Hampstead and Kilburn
22928 Bristol West
23448 Hornsey and Wood Green


In [36]:
conversions = []
with open("wards_to_lads.csv") as csv_file:
    reader = csv.reader(csv_file)
    for ii, line in enumerate(reader):
        if ii != 0:
            ward = line[0]
            district = line[1]
            conversions.append((ward, district))

signatures_by_lad = {}
for (ward, count) in signatures_by_constituency.items():
    matches = 0
    for (ward_match, district) in conversions:
        if ward == ward_match:
            matches += 1
        if district not in signatures_by_lad:
            signatures_by_lad[district] = 0
    
    for (ward_match, district) in conversions:
        if ward == ward_match:
            signatures_by_lad[district] += count/matches

signature_list = [[k, v] for (k, v) in signatures_by_lad.items()]
signature_list.sort(key=lambda x: x[1])

# # tidying
# for constituency in signature_list:
#     if constituency[0] == "Na h-Eileanan Siar":
#         constituency[0] = "Eilean Siar"
        

print("Votes  Constituency")
for constituency in signature_list[:10]:
    print("%5i %s" % tuple(constituency[::-1]))
print("..... .....")
for constituency in signature_list[-10:]:
    print("%5i %s" % tuple(constituency[::-1]))

Votes  Constituency
  597 Na h-Eileanan Siar
  633 Orkney Islands
  739 Shetland Islands
  743 Ballymoney
  795 Moyle
 1093 Isles of Scilly
 1115 Ballymena
 1133 Cookstown
 1133 Magherafelt
 1174 Limavady
..... .....
38933 Southwark
38995 Manchester
39118 Brighton and Hove
39303 City of Edinburgh
39560 Hackney
46634 Leeds
46762 Lambeth
49307 Bristol, City of
50434 Wandsworth
53968 Birmingham


In [37]:
def mercator(p):
    radius = 6378137
    out = [0, 0]
    out[0] = np.pi * radius * p[0] / 180;
    out[1] = radius * np.log(np.tan((45 + p[1]/2)*np.pi/180));

    return out

In [38]:
# Load in Geo data

with open("geo_eng_lad.json") as json_file:
    lads = json.load(json_file) 

feature_collection = lads['features']
feature = feature_collection[0]

# convert to Mercator
for feature in feature_collection:
    p = feature['geometry']
    if p['type'] == "Polygon":
        for polygon in p['coordinates']:
            for point in polygon:
                point[:] = mercator(point)
    
    elif p['type'] == "MultiPolygon":
        for polygons in p['coordinates']:
            for polygon in polygons:
                for point in polygon:
                    point[:] = mercator(point)



In [39]:
min_votes = signature_list[1][1]
max_votes = signature_list[-1][1]
def get_color(votes):
    c = (votes)/(max_votes)
    return (1, 1, 1 - c)

# Convert to patches

patches = []
names = []
minx = np.Inf
miny = np.Inf
maxx = -np.Inf
maxy = -np.Inf


key_exists = []

england_names = []
for feature in feature_collection:
    name = feature['properties']['LAD13NM']
    if name == "Eilean Siar": name = "Na h-Eileanan Siar"
    england_names.append(name)
    p = shape(feature['geometry'])
    try:
        votes = signatures_by_lad[name]
        key_exists.append(name)
    except KeyError:
        votes = 0
    if isinstance(p, MultiPolygon):
        for poly in p:
            lminx, lminy, lmaxx, lmaxy = p.bounds
            if lminx < minx: minx = lminx
            if lminy < miny: miny = lminy
            if lmaxx > maxx: maxx = lmaxx
            if lmaxy > maxy: maxy = lmaxy
            patches.append(PolygonPatch(poly, fc=get_color(votes), ec='#555555', lw=0.2, alpha=1., zorder=1))
    else:
        patches.append(PolygonPatch(p, fc=get_color(votes), ec='#555555', lw=0.2, alpha=1., zorder=1))
        lminx, lminy, lmaxx, lmaxy = p.bounds
        if lminx < minx: minx = lminx
        if lminy < miny: miny = lminy
        if lmaxx > maxx: maxx = lmaxx
        if lmaxy > maxy: maxy = lmaxy


In [40]:
signature_list = [[k, v] for (k, v) in signatures_by_lad.items() if k in england_names]
signature_list.sort(key=lambda x: x[1])
        

print("Votes  Constituency")
for constituency in signature_list[:10]:
    print("%5i %s" % tuple(constituency[::-1]))
print("..... .....")
for constituency in signature_list[-10:]:
    print("%5i %s" % tuple(constituency[::-1]))

Votes  Constituency
 1093 Isles of Scilly
 1195 Boston
 1723 Barrow-in-Furness
 1824 Tamworth
 1903 Bolsover
 1955 Hartlepool
 1998 Corby
 2130 West Somerset
 2198 Burnley
 2241 Forest Heath
..... .....
38325 Barnet
38933 Southwark
38995 Manchester
39118 Brighton and Hove
39560 Hackney
46634 Leeds
46762 Lambeth
49307 Bristol, City of
50434 Wandsworth
53968 Birmingham


In [41]:
fig = plt.figure()
ax = fig.add_subplot(111)

w, h = maxx - minx, maxy - miny
xmin, xmax = minx - 0.2 * w, maxx + 0.2 * w
ymin, ymax = miny - 0.2 * h, maxy + 0.2 * h
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])
ax.set_aspect(1)

ax.add_collection(PatchCollection(patches, match_original=True))
ax.set_xticks([])
ax.set_yticks([])

#scale
delta = h/20
x = 0.8*delta
ax.add_patch(Rectangle((xmin + w, ymin + 5*delta), x, x, fc=get_color(50000)))
ax.text(xmin + w + delta, ymin + 5*delta, "50000+")
ax.add_patch(Rectangle((xmin + w, ymin + 4*delta), x, x, fc=get_color(30000)))
ax.text(xmin + w + delta, ymin + 4*delta, "30000")
ax.add_patch(Rectangle((xmin + w, ymin + 3*delta), x, x, fc=get_color(10000)))
ax.text(xmin + w + delta, ymin + 3*delta, "10000")
ax.add_patch(Rectangle((xmin + w, ymin + 2*delta), x, x, fc=get_color(1000)))
ax.text(xmin + w + delta, ymin + 2*delta, "1000")
ax.add_patch(Rectangle((xmin + w, ymin + 1*delta), x, x, fc=get_color(0)))
ax.text(xmin + w + delta, ymin + 1*delta, "0")

ax.text(xmin + 0.2*w, ymax - 0.2*w, "Petition Signatures")
ax.text(xmin + delta, ymin + delta, "N. Lampe, 2016", color="gray", fontsize="x-small")

fig.savefig("petition.png", bbox_inches="tight", dpi=300)
fig.savefig("petition.svg", bbox_inches="tight")

In [42]:
# load election data

with open("referendum.json") as referendum_file:
    ref = json.load(referendum_file)
    
def get_district(name):
    for district in ref:
        if district['name'] == name:
            d = {}
            d['electorate'] = district['electorate']
            d['name'] = district['name']
            for answer in district['answers']:
                if answer['shortText'] == "Remain":
                    d['remain'] = answer['votes']
                else:
                    d['leave'] = answer['votes']
            return d

england_votes = {'leave': 0, 'remain': 0, 'electorate': 0}
for name in england_names:
    district = get_district(name)
    england_votes['remain'] += district['remain']
    england_votes['leave'] += district['leave']
    england_votes['electorate'] += district['electorate']

In [43]:
bregret_list = [[name, 100*signatures_by_lad[name]/get_district(name)['electorate']] for name in england_names]
bregret_list.sort(key=lambda x: x[1])

print("Bregret  Constituency")
for constituency in bregret_list[:10]:
    print("%7f %s" % tuple(constituency[::-1]))
print("..... .....")
for constituency in bregret_list[-10:]:
    print("%7f %s" % tuple(constituency[::-1]))

Bregret  Constituency
2.779476 Hartlepool
2.851743 Ashfield
2.866470 Redcar and Cleveland
3.036354 Boston
3.039595 Cannock Chase
3.068490 Blackpool
3.117577 North East Lincolnshire
3.140437 Doncaster
3.160928 Rotherham
3.178408 Mansfield
..... .....
22.411624 Hammersmith and Fulham
22.524584 Richmond upon Thames
22.975091 Wandsworth
23.718662 Haringey
24.227726 Hackney
25.368339 Camden
26.039569 Islington
30.560388 Kensington and Chelsea
60.800445 Isles of Scilly
220.586130 City of London


In [44]:
def get_color_pct(frac):
    c = frac/0.3
    if c > 1: c = 1
    return (1, 1, 1 - c)

# Convert to patches

patches = []
names = []
minx = np.Inf
miny = np.Inf
maxx = -np.Inf
maxy = -np.Inf


key_exists = []

for feature in feature_collection:
    name = feature['properties']['LAD13NM']
    if name == "Eilean Siar": name = "Na h-Eileanan Siar"
    population = get_district(name)["electorate"]
    p = shape(feature['geometry'])
    try:
        votes = signatures_by_lad[name]
        key_exists.append(name)
    except KeyError:
        votes = 0
    if name == "City of London": votes = 0
    if isinstance(p, MultiPolygon):
        for poly in p:
            lminx, lminy, lmaxx, lmaxy = p.bounds
            if lminx < minx: minx = lminx
            if lminy < miny: miny = lminy
            if lmaxx > maxx: maxx = lmaxx
            if lmaxy > maxy: maxy = lmaxy
            patches.append(PolygonPatch(poly, fc=get_color_pct(votes/population), ec='#555555', lw=0.2, alpha=1., zorder=1))
    else:
        patches.append(PolygonPatch(p, fc=get_color_pct(votes/population), ec='#555555', lw=0.2, alpha=1., zorder=1))
        lminx, lminy, lmaxx, lmaxy = p.bounds
        if lminx < minx: minx = lminx
        if lminy < miny: miny = lminy
        if lmaxx > maxx: maxx = lmaxx
        if lmaxy > maxy: maxy = lmaxy

In [45]:
fig = plt.figure()
ax = fig.add_subplot(111)

w, h = maxx - minx, maxy - miny
xmin, xmax = minx - 0.2 * w, maxx + 0.2 * w
ymin, ymax = miny - 0.2 * h, maxy + 0.2 * h
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])
ax.set_aspect(1)

ax.add_collection(PatchCollection(patches, match_original=True))
ax.set_xticks([])
ax.set_yticks([])

#scale
delta = h/20
x = 0.8*delta
ax.add_patch(Rectangle((xmin + w, ymin + 5*delta), x, x, fc=get_color_pct(0.3)))
ax.text(xmin + w + delta, ymin + 5*delta, "30%")
ax.add_patch(Rectangle((xmin + w, ymin + 4*delta), x, x, fc=get_color_pct(0.2)))
ax.text(xmin + w + delta, ymin + 4*delta, "20%")
ax.add_patch(Rectangle((xmin + w, ymin + 3*delta), x, x, fc=get_color_pct(0.1)))
ax.text(xmin + w + delta, ymin + 3*delta, "10%")
ax.add_patch(Rectangle((xmin + w, ymin + 2*delta), x, x, fc=get_color_pct(0.05)))
ax.text(xmin + w + delta, ymin + 2*delta, "5%")
ax.add_patch(Rectangle((xmin + w, ymin + 1*delta), x, x, fc=get_color_pct(0)))
ax.text(xmin + w + delta, ymin + 1*delta, "0%")

ax.text(xmin + 0.2*w, ymax - 0.2*w, "Bregretfulness Index")
ax.text(xmin + delta, ymin + delta, "N. Lampe, 2016", color="gray", fontsize="x-small")

fig.savefig("bregret.png", bbox_inches="tight", dpi=300)
fig.savefig("bregret.svg", bbox_inches="tight")

In [46]:
# Breget differential
total_english_petition_votes = sum([signatures_by_lad[name] for name in england_names])
bregret_differential = {}
scaling = england_votes['remain']/total_english_petition_votes
print(scaling)
for name in england_names:
    district = get_district(name)
    remain_votes = district['remain']
    petition_signatures = signatures_by_lad[name]
    expected_signatures = petition_signatures*scaling
    bregret_differential[name] = 100*(expected_signatures - remain_votes)/remain_votes
    

bregret_diff_list = [[k,v] for (k,v) in bregret_differential.items()]
bregret_diff_list.sort(key=lambda x: x[1])

print("Bregret (%)  District")
for constituency in bregret_diff_list[:10]:
    print("%6.1f       %s" % tuple(constituency[::-1]))
print(".....        .....")
for constituency in bregret_diff_list[-10:]:
    print("%6.1f       %s" % tuple(constituency[::-1]))



3.928344497721161
Bregret (%)  District
 -56.0       Hambleton
 -52.6       Redcar and Cleveland
 -52.4       Barrow-in-Furness
 -49.0       Ashfield
 -47.7       Tamworth
 -46.3       Cannock Chase
 -45.3       Hartlepool
 -45.0       North Lincolnshire
 -44.9       Nuneaton and Bedworth
 -44.4       Rotherham
.....        .....
  67.2       Wandsworth
  74.8       Haringey
  79.9       Hammersmith and Fulham
  86.3       Hackney
  91.4       Tower Hamlets
  93.4       Islington
 103.1       Camden
 165.1       Kensington and Chelsea
 435.1       Isles of Scilly
1466.2       City of London


In [47]:
# plot bregret

def get_color_differential(pct):
    if pct > 50: pct = 50
    if pct < -50: pct = -50
    if pct < 0:
        c = -pct/50
        return (1 - c, 1 - c, 1)
    if pct >= 0:
        c = pct/50
        return (1, 1, 1 - c)

# Convert to patches

patches = []
names = []
minx = np.Inf
miny = np.Inf
maxx = -np.Inf
maxy = -np.Inf


key_exists = []

for feature in feature_collection:
    name = feature['properties']['LAD13NM']
    if name == "Eilean Siar": name = "Na h-Eileanan Siar"
    population = get_district(name)["electorate"]
    p = shape(feature['geometry'])
    try:
        votes = signatures_by_lad[name]
        bregret = bregret_differential[name]
    except KeyError:
        bregret = 0
    if isinstance(p, MultiPolygon):
        for poly in p:
            lminx, lminy, lmaxx, lmaxy = p.bounds
            if lminx < minx: minx = lminx
            if lminy < miny: miny = lminy
            if lmaxx > maxx: maxx = lmaxx
            if lmaxy > maxy: maxy = lmaxy
            patches.append(PolygonPatch(poly, fc=get_color_differential(bregret), ec='#555555', lw=0.2, alpha=1., zorder=1))
    else:
        patches.append(PolygonPatch(p, fc=get_color_differential(bregret), ec='#555555', lw=0.2, alpha=1., zorder=1))
        lminx, lminy, lmaxx, lmaxy = p.bounds
        if lminx < minx: minx = lminx
        if lminy < miny: miny = lminy
        if lmaxx > maxx: maxx = lmaxx
        if lmaxy > maxy: maxy = lmaxy

In [48]:
fig = plt.figure()
ax = fig.add_subplot(111)

w, h = maxx - minx, maxy - miny
xmin, xmax = minx - 0.2 * w, maxx + 0.2 * w
ymin, ymax = miny - 0.2 * h, maxy + 0.2 * h
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])
ax.set_aspect(1)

ax.add_collection(PatchCollection(patches, match_original=True))
ax.set_xticks([])
ax.set_yticks([])

#scale
delta = h/20
x = 0.8*delta
ax.add_patch(Rectangle((xmin + w, ymin + 5*delta), x, x, fc=get_color_differential(50)))
ax.text(xmin + w + delta, ymin + 5*delta, "50%")
ax.add_patch(Rectangle((xmin + w, ymin + 4*delta), x, x, fc=get_color_differential(20)))
ax.text(xmin + w + delta, ymin + 4*delta, "20%")
ax.add_patch(Rectangle((xmin + w, ymin + 3*delta), x, x, fc=get_color_differential(0)))
ax.text(xmin + w + delta, ymin + 3*delta, "0%")
ax.add_patch(Rectangle((xmin + w, ymin + 2*delta), x, x, fc=get_color_differential(-20)))
ax.text(xmin + w + delta, ymin + 2*delta, "20%")
ax.add_patch(Rectangle((xmin + w, ymin + 1*delta), x, x, fc=get_color_differential(-50)))
ax.text(xmin + w + delta, ymin + 1*delta, "-50%")

ax.text(xmin + 0.2*w, ymax - 0.2*w, "Bregret Difference")
ax.text(xmin + delta, ymin + delta, "N. Lampe, 2016", color="gray", fontsize="x-small")

fig.savefig("bregret_diff.png", bbox_inches="tight", dpi=300)
fig.savefig("bregret_diff.svg", bbox_inches="tight")

In [49]:
print("Total Petition Signatures: " + str(total_petition_votes))
print("Count from Wards: " + str(int(sum([v for v in signatures_by_constituency.values()]))))
print("Count from LADs: " + str(int(sum([v for v in signatures_by_lad.values()]))))
print("Total English Petition Signatures: " + str(int(total_english_petition_votes)))
print("English Remain Votes: " + str(england_votes["remain"]))
print("English Leave Votes: " + str(england_votes["leave"]))

Total Petition Signatures: 3779528
Count from Wards: 3779528
Count from LADs: 3779527
Total English Petition Signatures: 3372329
English Remain Votes: 13247674
English Leave Votes: 15187583
